# SSLT-E2K: Sequence to Sequence Language Translation from English to Konkani using Natural Language Processing

#### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import keras
import pickle
import tensorflow
import nltk
from nltk.translate.bleu_score import sentence_bleu
from keras.models import load_model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

#### Importing Dataset

In [ ]:
lines= pd.read_csv('final_dataset.csv', names=['eng', 'konk'], header=None)

lines.head(10)

FileNotFoundError: ignored

In [ ]:
lines.shape

#### Dataset Per-processing

In [ ]:
# Lowercase all characters
lines.eng = [x.lower() for x in lines.eng]

In [ ]:
# Remove all numbers from text
lines.eng = [re.sub("[1234567890]","",x) for x in lines.eng]
lines.eng = [re.sub("[\u200d]","",x) for x in lines.eng]
lines.konk = [re.sub("[1234567890२३०८१५७९४६]","",x) for x in lines.konk]
lines.konk = [re.sub("[\u200d]","",x) for x in lines.konk]

In [ ]:
# Remove special character
lines.eng = [re.sub("`",'',x) for x in lines.eng]
lines.konk = [re.sub("`",'',x) for x in lines.konk]

In [ ]:
# Remove quotes
lines.eng = [re.sub("'",'',x) for x in lines.eng]
lines.konk = [re.sub("'",'',x) for x in lines.konk]
lines.eng = [re.sub("‘",'',x) for x in lines.eng]
lines.konk = [re.sub("‘",'',x) for x in lines.konk]
lines.eng = [re.sub("’",'',x) for x in lines.eng]
lines.konk = [re.sub("’",'',x) for x in lines.konk]

In [ ]:
# Remove extra spaces
lines.eng = [x.strip() for x in lines.eng]
lines.konk = [x.strip() for x in lines.konk]

In [ ]:
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.konk=lines.konk.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Add start and end tokens to target sequences
lines.konk = ['START_ '+ x + ' _END' for x in lines.konk]

In [ ]:
lines.sample(10)

,eng,konk
16283,there is a taxi facility to go to neelkanth,START_ निलकंठाक वचपा खातीर टॅक्सींची वेवस्था आ...
54517,the mahamastakabhisheka held in and the compl...,START_ वर्सा आयोजीत महामस्तकाभिशेक आनी मुर्तीच...
15310,some fatty substances or oils may also be given,START_ कांय चरबी आशिल्ले पदार्थ वा तेल लेगीत द...
4260,take time to meditate in solitude,START_ एकांतांत ध्यान मेडिटेशन करपा खातीर वे...
45401,popularly known as domuhani this place is a pi...,START_ दोमुहानी नांवान लोकप्रीय ही सुवात खडकई ...
50438,deshsewa and was dedicated then press and jour...,START_ देशसेवा आनी जनरुचीचे शुध्दीकरणाचे म्हान...
10092,large pandals are made from place to place,START_ जाग्या जाग्यार व्हड व्हड माटव घालतात _END
27992,gujarati hindi english urdu and portuguese are...,START_ गुजराती हिंदी इंग्लेज उर्दू आनी पुर्...
57068,they do not face any kind of trouble even on l...,START_ उणी न्हीद घेवन लेगीत तांकां कसलेच तरेचे...
46469,this says salman i definitely want to but wil...,START_ हाचेर सलमान म्हणटा निश्चीतच जाय पूण खं...


#### Creating English-Konknai Vocabulary

In [ ]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Konkani 
all_konkani_words=set()
for konk in lines.konk:
    for word in konk.split():
        if word not in all_konkani_words:
            all_konkani_words.add(word)

In [ ]:
# Max Length of source sequence
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(' ')))
max_length_src = np.max(length_list)
max_length_src

28

In [ ]:
# Max Length of target sequence
length_list=[]
for l in lines.konk:
    length_list.append(len(l.split(' ')))
max_length_tar = np.max(length_list)
max_length_tar

52

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_konkani_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_konkani_words)
num_encoder_tokens, num_decoder_tokens

(38157, 89434)

In [ ]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

89435

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,eng,konk
35887,anemia medication is given to reduce bleeding ...,START_ रक्तस्राव उणो करपाचें वा दूख उणावपाचें ...
56690,the birds found in the great himalayan nationa...,START_ ग्रेट हिमालयन राष्ट्रीय उद्यानांत मेळपी...
32732,in blind piles there are no warts on the outsi...,START_ वाये पसून जाल्ल्या बिकांत भोंकाचे भायले...
31519,also both actors is hosted jointly by the fift...,START_ हाचे खेरीज दोनांय अभिनेत्यांनी बिग बॉस...
15449,tagore knew no english no one knows indian eng...,START_ टागोराक इंग्लीश कळना खंयच्याच भारतियाक ...
45722,take the melon pulp and rub it gently on the a...,START_ काळंगाचो गर घेयात आनी ताका ब्लॅक हेड्स...
60813,village by the ministry of agriculture the vil...,START_ कृषी मंत्रालयाच्या वतीन गांवागांवानी गु...
18933,it is not possible to cultivate it in unirriga...,START_ शिंपूंक नाशिल्ल्या क्षेत्रांनी हाची शेत...
23256,good luck at a reasonable price is typical of ...,START_ योग्य दरेन बरो थातोमातो हें गणगौर पॅलेस...
12950,behu singh ji were a contemporary buffoon mast...,START_ बेहू सिंहाचो एक समकालीन सवंग गुरू मूलरा...


#### Defining Train-Test Data

In [ ]:
# Train - Test Split (80-20)
X, y = lines.eng, lines.konk
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape

((52428,), (13108,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 256):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            print(encoder_input_data)
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

#### Encoder - Decoder Model Architecture

In [ ]:
latent_dim = 100

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens +1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens +1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, dropout = 0.5, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 256
epochs = 110

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    3815800     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    8943600     input_2[0][0]                    
______________________________________________________________________________________________

#### Training the model

In [ ]:
history=model.fit\(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)


Epoch 1/125
204/204 [==============================] - 1485s 7s/step - loss: 2.3239 - acc: 0.0758 - val_loss: 1.9739 - val_acc: 0.0934
Epoch 2/125
204/204 [==============================] - 1461s 7s/step - loss: 1.9273 - acc: 0.0969 - val_loss: 1.9362 - val_acc: 0.1027
Epoch 3/125
204/204 [==============================] - 1492s 7s/step - loss: 1.8799 - acc: 0.1033 - val_loss: 1.9228 - val_acc: 0.1058
Epoch 4/125
204/204 [==============================] - 1460s 7s/step - loss: 1.8485 - acc: 0.1073 - val_loss: 1.9158 - val_acc: 0.1077
Epoch 5/125
204/204 [==============================] - 1495s 7s/step - loss: 1.8239 - acc: 0.1113 - val_loss: 1.9009 - val_acc: 0.1123
Epoch 6/125
204/204 [==============================] - 1459s 7s/step - loss: 1.7933 - acc: 0.1171 - val_loss: 1.8798 - val_acc: 0.1168
Epoch 7/125
204/204 [==============================] - 1463s 7s/step - loss: 1.7574 - acc: 0.1249 - val_loss: 1.8550 - val_acc: 0.1265
Epoch 8/125
204/204 [==============================] - 

#### Always remember to save the weights and model

In [ ]:
model.save_weights('weights_ss.h5')
print("Saved weights to disk")

Saved weights to disk


In [ ]:
history = model.save('model_s.h5')
print("Saved model to disk")

Saved model to disk


In [ ]:
Pkl_Filename = "model_s.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(history, file)

In [ ]:
# Pickled_history = pickle.load(open('model_s.pkl','rb'))

In [ ]:
# # save:
# f = open('model.pckl', 'wb')
# pickle.dump(history.history, f)
# f.close()

# # retrieve:    
# f = open('model.pckl', 'rb')
# history = pickle.load(f)
# f.close()

AttributeError: 'NoneType' object has no attribute 'history'

#### Load the weights, if you close the application

In [ ]:
model.load_weights('weights_ss.h5')

In [ ]:
plt.plot(history.load['val_loss'])

#### Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

#### Decode sample sequeces

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Evaluation on Train Dataset

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
#1
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: these states are less onion wholesale prices
Actual Konkani Translation:  ह्या राज्यांनीय कांद्याचे ठोक भाव कमी जाल्यांत 
Predicted Konkani Translation:  ह्या राज्यांनीय कांद्याचे दर वर्सा विकसीत केलें 


In [ ]:
#2
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#3
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#4
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#5
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#6
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#7
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#8
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#9
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

In [ ]:
#10
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Konkani Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

#### Evaluation on Validation Dataset

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
#1
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: workers working in cotton mills or plastic factories in industrial areas are susceptible to chronic bronchitisdisease
Actual Konkani Translation:  उद्येगीक वाठारांनी कापसाची गिरण वा प्लास्टिकाच्या कारखान्यांत काम करप्यांक कामगारांक  क्रॉनीक ब्रॉन्कायटीस  दुयेंस जावपाची शक्यताय आसता 
Predicted Konkani Translation:  रेंवटाच्या वाठारा वांगडा इंडियन कुमार आनी केबल


In [ ]:
#2
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: so the country through the platform in swangon ends the question of introduction of the period
Actual Konkani Translation:  देखून ह्या सवंगानी मंचाच्या माध्यमांतल्यान देश काळाचो परीचय दिवपाचो प्रस्नच सोंपता 
Predicted Konkani Translation:  देखून कृशी दसकांत ग्रामीण अर्थवेवस्थेंत पुराय 


In [ ]:
#3
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: camps can be set up here during the summer on the spring so that the natural climate and climate of the surroundings can be studied
Actual Konkani Translation:  वसंत मोसमाचे सुरवातीक कालोराच्या दिसांनी हांगा तंबू घालूं येता जाचे आशिकुशीचे सैमीक रानवटी वाठाराचो आनी हवामानाचो अभ्यास करूं येता 
Predicted Konkani Translation:  नव्या वाठारांत तुमच्यांनी सैमीक देखावे मनाक आग


In [ ]:
#4
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: snow bridges tend to be unreliable due to the weakening of the day climbing
Actual Konkani Translation:  बर्फाचेर तयार जाल्ले पूल दीस वयर आयले उपरांत दुबळे पडिल्ल्या कारणान घातकी जावंक लागतात 
Predicted Konkani Translation:  काठार एक्सपर्ट लोक जे दुयेंतीच्या कॅन्सराक पांयां


In [ ]:
#5
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: during makar sankranti parashurama goes to take a dip in the sailani kund from far and wide
Actual Konkani Translation:  मकर सक्रांती वेळार परशुराम कुंडांत पयसुल्ल्यान भोंवडेकार कुंडांत न्हावपा खातीर येतात 
Predicted Konkani Translation:  डिसेंबराचे डिसेंबराचे नाण्यांचो उपेग ट्रेल्स कुं


In [ ]:
#6
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: this same information and comparison sometimes helps us in getting out of our grief
Actual Konkani Translation:  हीच म्हायती आनी तुळा जायत्या फावटीं आमी आमच्या दुख्खांतल्यान भायर सरपांत आमची मजत करता 
Predicted Konkani Translation:  हे खातीर सुचना साम्राज्यवाद आतां जायत्या तरांच्यो तप


In [ ]:
#7
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: round the same texts references to a game of sports name
Actual Konkani Translation:  त्याच ग्रंथांत गोल क्रिडा नांवाचो एका खेळाचो उल्लेख जाला 
Predicted Konkani Translation:  होंवरे उपरांत राधा गुरू शेख दाऊद रावताले 


In [ ]:
#8
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: listen to it too that harry had broken away from the heart of the model of the beautiful taylor they had left him
Actual Konkani Translation:  आयकुपाक तर हे लेगीत येता की हॅरीन सुंदर टेलराचे काळीज ज्या मॉडला खातीर तोडिल्लें ताणें तिका पसून सोडली 
Predicted Konkani Translation:  ह्या देंवण्या लागीं सावन जेन्ना सावन अकरमा कडेन 


In [ ]:
#9
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: large retail marketing areas india is the ninth place
Actual Konkani Translation:  व्हड किरकोळ वेपार धंदो आशिल्ल्या क्षेत्रांनी भारताक णव्वी सुवात प्राप्त जाल्या 
Predicted Konkani Translation:  भारतांत भारतांत उसाची शेती भारतांतल्या एक विशाळ क


In [ ]:
#10
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Konkani Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Konkani Translation:', decoded_sentence[:-4])

Input English sentence: the havelis of jaisalmer which are its specialty are made of bright yellow stones
Actual Konkani Translation:  जैसलमेराचे वाडे जें हाचें खाशेलपण आसा  चकचकीत हळडुव्या फातरांनी बांदिल्ले आसात 
Predicted Konkani Translation:  आधुनीक नदरेन जैन देवळाक दरगो आसा जें शत्रूं पसून वाट


---